# SmoothLife

## A first description of gliders in continuous CA 

<!-- Image link to be used after making repo public -->
<!-- <img src="https://raw.github.com/riveSunder/DiscoGliders/assets/smoothlife_glider_summary.png"> -->


### Overview

<!-- local image -->
![Smoothlife summary](../assets/smoothlife_glider_summary.png)

The description of SmoothLife by [Rafler in 2011](https://arxiv.org/abs/1111.1567) was accompanied by the first description of a glider in a continuous cellular automaton, complementing the earlier discovery of gliders in the Gray-Scott reaction diffusion system, [U-Skate World](https://www.mrob.com/pub/comp/xmorphia/catalog.html).

We can distill SmoothLife dynamics into the form of the Euler method: the next state of cell values $A_{t+\Delta t}$ is the current cell states $A_t$ plus the results of an update funciton $U(A_t) = \frac{\partial A}{\partial t}$ weighted by a temporal step size $\Delta t$.

$$
A_{t+ \Delta t} = A_t + \Delta t * U(A_t)
$$

Additionally, the cell state values are squashed or truncated to values between 0 and 1.0, denoted by $[\cdot]_0^1$ below:

$$
A_{t+ \Delta t} = [A_t + \Delta t * U(A_t)]_0^1
$$


### Neighborhoods

Unlike [Lenia](./lenia.ipynb) and [glaberish](glaberish.ipynb) CA frameworks, SmoothLife was designed to use inner and outer neighborhood kernels. These kernels have hard boundaries, or they can be optionally smoothed/apodized as suggested by ([Rafler](https://arxiv.org/abs/1111.1567).

The inner neighborhood kernel has a value of $\frac{1}{M}$ at every position within an inner radius $r_i$, where $M$ is the sum of cells within the inner radius. 

$$
K_i = \left\{ 
        \begin{array}{ c l }
            \frac{1}{M} & \quad \textrm{if } r < r_i \\
            0                 & \quad \textrm{otherwise}
            \end{array}
            \right.
$$

Similarly, the outer neighborhood has non-zero value $\frac{1}{N}$ between the inner radius and the outer radius $r_o$, where $N$ is the sum of cells between those radii. 

$$
K_n = \left\{ 
        \begin{array}{ c l }
            \frac{1}{N} & \quad \textrm{if } r_i \leq r < r_o\\
            0                 & \quad \textrm{otherwise}
            \end{array}
            \right.
$$

### Updates

SmoothLife updates comprise smooth intervals composed of shifted sigmoids:

$$
s_{\mu, \alpha}(x) = \frac{1}{1+e^{-\frac{x-\mu}{\alpha}}}
$$

Sigmoid $s_{\mu, \alpha}$ has a smooth 'edge' at $\mu$, the smoothness of which is determined by variable $\alpha$. 

My implementation of SmoothLife fits into the [glaberish](glaberish.ipynb) framework. The genesis function is a smooth interval with upper and lower 'edges' at $\mu_{gl}$ and $\mu_{gu}$, respectively. 

$$
G(x) = 2 * s_{\mu_{gl}, \alpha_g}(x) * (1- s_{\mu_{gu}}) - 1
$$

The persistence function is a smooth interval with upper and lower 'edges' at $\mu_{pl}$ and $\mu_{pu}$, respectively. 

$$
P(x) = 2 * s_{\mu_{pl}, \alpha_p}(x) * (1- s_{\mu_{pu}}) - 1
$$

The genesis function is applied to the inner neighborhood $K_i \circledast A_t$, and the persistence function is applied to the outer neighborhood $K_n \circledast A_t$.

I use only one set of parameters for SmoothLife in this work: $\alpha_g=0.0280$, $\alpha_p=0.1470$, $\mu_{gl}=0.2780$, $\mu_{gu}=0.3650$, $\mu_{pl}=0.2670$, and $\mu_{pu}=0.4450$


<!-- Image link to be used after making repo public -->
<!-- <img src="https://raw.github.com/riveSunder/DiscoGliders/assets/smoothlife_4.gif"> -->

<!-- local image -->
![SmoothLife gliders](../assets/smoothlife_4.gif)

In [ ]:
import os
import time

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams["animation.embed_limit"] = 128

import skimage
import skimage.io as sio
import skimage.transform

import yuca
from yuca.ca.neural import NCA
from yuca.ca.continuous import CCA
from yuca.cppn import CPPN

from yuca.zoo.librarian import Librarian
from yuca.kernels import get_kernel

torch.set_default_dtype(torch.float32)

import IPython

from importlib import reload
reload(yuca)
reload(yuca.ca)

In [ ]:
def plot_grid(grid, my_cmap=plt.get_cmap("magma"), title="SmoothLife animation", vmin=0.0, vmax=1):

    global subplot_0
    
    fig, ax = plt.subplots(1,1, figsize=(4.5,4.5), facecolor="white")

    # TODO invert cmap
    
    my_cmap=plt.get_cmap("magma")
    grid_display = 1.0 - my_cmap(np.array(grid[0,0]))[:,:,:3]
    
    subplot_0 = ax.imshow(grid_display, interpolation="nearest")
    
    fig.suptitle(title, fontsize=8)

    ax.set_yticklabels('')
    ax.set_xticklabels('')
    
    plt.tight_layout()

    return fig, ax

def update_fig(i):

    global subplot_0    
    global grid
    #global ax
    
    grid = ca(grid)
    
    my_cmap=plt.get_cmap("magma")
    grid_display = 1.0 - my_cmap(np.array(grid[0,0].cpu()))[:,:,:3]
    
    subplot_0.set_array(grid_display)
        
    plt.tight_layout()
    
def scale_pattern(scale_h, scale_w, pattern):
    
    use_anti_aliasing = True
    
    scaled_pattern = skimage.transform.rescale(pattern, \
            scale=(1,1, scale_h, scale_w), \
            anti_aliasing=use_anti_aliasing,\
            mode="constant", cval=0.0)
    
    return scaled_pattern

In [ ]:
lib = Librarian(verbose=False)

save_figs = False
my_device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

num_frames = 256
grid_dim = 96

In [ ]:
pattern_name = "smoothlife_single_glider000"


p, m = lib.load(pattern_name)
ca = CCA()
ca.restore_config(m["ca_config"])
ca.set_dt(0.85)

grid = ca.initialize_grid(dim=grid_dim)

grid[:,:,:p.shape[-2],:p.shape[-1]] = torch.tensor(p)

grid[:,:,-p.shape[-2]:,-p.shape[-1]:] = torch.tensor(p.transpose(0,1,3,2))

fig, ax = plot_grid(grid)
plt.show()

if save_figs:
    save_path = os.path.join("..","assets", "smoothlife_4.gif")
    matplotlib.animation.FuncAnimation(fig, update_fig, frames=num_frames, interval=10).save(save_path)
    
IPython.display.HTML(\
        matplotlib.animation.FuncAnimation(fig, update_fig, frames=num_frames, interval=10).to_jshtml())

In [ ]:
"""
After a warm-up period, 
The SmoothLife glider is stable for ~1024 steps with $K_r = 36$, $\Delta t=0.19$, and 16-bit PyTorch floats,
but unstable by 1024 steps with $K_r = 36$, $\Delta t=0.19$, and 32 or 64-bit PyTorch floats,
"""

In [ ]:
number_steps = 1024
my_radius = 36
my_dt = 0.225
warmup_steps = 192

pattern_name = "smoothlife_single_glider000"
p, m = lib.load(pattern_name)
ca = CCA()

ca.restore_config(m["ca_config"])
ca.no_grad()
ca.to_device(my_device)
native_radius = ca.kernel_radius
my_scale = my_radius/native_radius

ca.set_dt(my_dt)
ca.set_kernel_radius(my_radius)

p = scale_pattern(my_scale, my_scale, p)

grid_dim = int(96 * my_scale)

grid = ca.initialize_grid(dim=grid_dim).to(my_device)
grid[:,:,32:32+p.shape[-2],32:32+p.shape[-1]] = torch.tensor(p)

fig, ax = plot_grid(grid.cpu())
fig.suptitle("initial grid", fontsize=24)

for my_step in range(warmup_steps):
    grid = ca(grid)
    
fig, ax = plot_grid(grid.cpu())
fig.suptitle(f"grid after {warmup_steps} warmup steps", fontsize=24)

grid_0 = 1.0 * grid

In [ ]:
my_dt = 0.19

In [ ]:
if torch.cuda.is_available():
    torch.set_default_dtype(torch.float16)
else:
    torch.set_default_dtype(torch.float32)

pattern_name = "smoothlife_single_glider000"
p, m = lib.load(pattern_name)
ca = CCA()
ca.restore_config(m["ca_config"])
ca.no_grad()
ca.to_device(my_device)

grid = 1.0 * grid_0.to(torch.get_default_dtype())

ca.set_dt(my_dt)
ca.set_kernel_radius(my_radius)

fig, ax = plot_grid(grid.cpu())
fig.suptitle("", fontsize=24)

IPython.display.HTML(\
        matplotlib.animation.FuncAnimation(fig, update_fig, frames=num_frames, interval=10).to_jshtml())

In [ ]:
torch.set_default_dtype(torch.float32)

pattern_name = "smoothlife_single_glider000"
p, m = lib.load(pattern_name)
ca = CCA()
ca.restore_config(m["ca_config"])
ca.no_grad()
ca.to_device(my_device)

grid = 1.0 * grid_0.to(torch.get_default_dtype())

ca.set_dt(my_dt)
ca.set_kernel_radius(my_radius)

fig, ax = plot_grid(grid.cpu())
fig.suptitle("initial grid", fontsize=24)
IPython.display.HTML(\
        matplotlib.animation.FuncAnimation(fig, update_fig, frames=num_frames, interval=10).to_jshtml())

In [ ]:
torch.set_default_dtype(torch.float64)

pattern_name = "smoothlife_single_glider000"
p, m = lib.load(pattern_name)
ca = CCA()
ca.restore_config(m["ca_config"])
ca.no_grad()
ca.to_device(my_device)

grid = 1.0 * grid_0.to(torch.get_default_dtype())

ca.set_dt(my_dt)
ca.set_kernel_radius(my_radius)

fig, ax = plot_grid(grid.cpu())
fig.suptitle("", fontsize=24)
IPython.display.HTML(\
        matplotlib.animation.FuncAnimation(fig, update_fig, frames=num_frames, interval=10).to_jshtml())